In [ ]:
Dataset: A folder structure like:

data/
  ├─ train/
  │    ├─ healthy/
  │    └─ diseased/
  └─ val/
       ├─ healthy/
       └─ diseased/

In [ ]:
Setup & Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models, optimizers
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
 Data Pipeline & Augmentation
img_size = (224, 224)
batch = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    zoom_range=0.2
)
val_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory('data/train', target_size=img_size, batch_size=batch, class_mode='binary')
val_ds = val_gen.flow_from_directory('data/val', target_size=img_size, batch_size=batch, class_mode='binary')



In [ ]:
Model Construction
base = MobileNetV2(input_shape=img_size + (3,), include_top=False, weights='imagenet')
base.trainable = False

model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
 Training

EPOCHS = 10

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds
)


In [ ]:
Plotting Training Curves:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend(), plt.show()


In [ ]:
Fine-Tuning (Optional)
base.trainable = True
fine_layers = 20
for layer in base.layers[:-fine_layers]:
    layer.trainable = False

model.compile(optimizer=optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, epochs=5, validation_data=val_ds)


In [ ]:
 Inference on New Images
def predict_leaf(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
    arr = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    pred = model.predict(np.expand_dims(arr, 0))[0][0]
    return 'Diseased' if pred > 0.5 else 'Healthy'

print(predict_leaf('sample_leaf.jpg'))
